# D-SPIN scRNA-seq preprocessing
Updated: 6/13/25.

This notebook assumes the existence of top-level `data/` and `output` directories. Please see the attached `environment.yml` file in order to reproduce the environment used for this analysis. 

In [2]:
import os
import matplotlib

import numpy as np
import anndata as ad
import scanpy as sc
import pandas as pd

%matplotlib inline
sc.set_figure_params(
    dpi=300
)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.use('cairo')

In [3]:
DATA_DIR = "../data/gbm43_perturb_seq"
OUTPUT_DIR = "../output/gbm43/dspin"

## Load data

In [ ]:
# Right now, we will load an h5ad object that contains
# raw RNA counts. Check out what this looks like.
gbm43_data = ad.read_h5ad(os.path.join(DATA_DIR, "gbm43.h5ad"))
gbm43_data

AnnData object with n_obs × n_vars = 18938 × 36601
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'sorted', 'cond', 'SCT_snn_res.0.4', 'seurat_clusters', 'sgRNA_UMI', 'sgRNA_logUMI', 'sgRNA_binary', 'sgRNA', 'sgRNACond'
    var: 'features'
    obsm: 'X_umap'

## Reprocess the data

In [5]:
from pathlib import Path
SAVE_PATH = Path("../output/gbm43") / "qc"
sc._settings.ScanpyConfig.figdir = SAVE_PATH

In [ ]:
# Unfortunately, SCTransform doesn't work for running oNMF because there are
# negative values produced. We will have to take the raw data, normalize it, and
# log transform it.
gbm43_data.var_names = gbm43_data.var_names.str.replace("^GRCh38-", "", regex=True)

In [ ]:
# Do QC
gbm43_data.var["mt"] = gbm43_data.var_names.str.startswith("MT-")
gbm43_data.var["ribo"] = gbm43_data.var_names.str.startswith("RPS", "RPL")
sc.pp.calculate_qc_metrics(
    gbm43_data, qc_vars=["mt", "ribo"], inplace=True, log1p=True
)
sc.pl.violin(
    gbm43_data,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
    save="_mito_and_total_counts.pdf"
)

In [ ]:
sc.pl.scatter(gbm43_data, "total_counts", "n_genes_by_counts", color="pct_counts_mt", save="_n_genes_by_counts_pct_counts_mt.pdf")

In [ ]:
# Filter permissively on mitochondrial characteristics
sc.pp.filter_cells(gbm43_data, min_genes=100)
sc.pp.filter_genes(gbm43_data, min_cells=3)

# Note that we could implement doublet detection later
gbm43_data

AnnData object with n_obs × n_vars = 18938 × 23511
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'sorted', 'cond', 'SCT_snn_res.0.4', 'seurat_clusters', 'sgRNA_UMI', 'sgRNA_logUMI', 'sgRNA_binary', 'sgRNA', 'sgRNACond', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes'
    var: 'features', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    obsm: 'X_umap'

In [ ]:
# Normalize and log transform the data
gbm43_data.layers["counts"] = gbm43_data.X.copy()
sc.pp.normalize_total(gbm43_data)
sc.pp.log1p(gbm43_data)

In [ ]:
# Get rid of the raw object so we don't have any confusion
gbm43_data.raw = None

In [ ]:
# Replace the cond 0s and 1s with RT/noRT
gbm43_data.obs["cond"] = gbm43_data.obs["cond"].replace({0: "noRT", 1: "RT"})

In [ ]:
gbm43_data_RT = gbm43_data[gbm43_data.obs["cond"] == "RT"].copy()
gbm43_data_noRT = gbm43_data[gbm43_data.obs["cond"] == "noRT"].copy()

In [ ]:
# As RT and noRT may have drastically different phenotypes, we want to include highly variable genes from each of them.
# Calculate the top 2000 highly variable genes for RT and noRT and take the union as our set of highly variable genes.
sc.pp.highly_variable_genes(gbm43_data_RT, n_top_genes=2000, batch_key="orig.ident")
sc.pp.highly_variable_genes(gbm43_data_noRT, n_top_genes=2000, batch_key="orig.ident")

hvg_RT = gbm43_data_RT.var[gbm43_data_RT.var['highly_variable']].index
hvg_noRT = gbm43_data_noRT.var[gbm43_data_noRT.var['highly_variable']].index

# Take the union of highly variable genes from both conditions
hvg_union = hvg_RT.union(hvg_noRT)

# Mark the highly variable genes in the original dataset
gbm43_data.var['highly_variable'] = gbm43_data.var_names.isin(hvg_union)


In [ ]:
# Find cell cycle scores
cell_cycle_genes = [x.strip() for x in open("../data/regev_lab_cell_cycle_genes.txt")]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
s_genes = [x for x in s_genes if x in gbm43_data.var_names]
g2m_genes = [x for x in g2m_genes if x in gbm43_data.var_names]
sc.tl.score_genes_cell_cycle(gbm43_data, s_genes=s_genes, g2m_genes=g2m_genes, use_raw=False)

In [ ]:
# Run PCA and clustering
sc.tl.pca(gbm43_data)

In [ ]:
# How many PCs should we consider?
sc.pl.pca_variance_ratio(gbm43_data, n_pcs=50, log=True, save="_pca_variance_ratio.pdf", show=False)

In [ ]:
sc.pl.pca(
    gbm43_data,
    color=["orig.ident", "orig.ident", "pct_counts_mt", "pct_counts_mt", "phase", "phase"],
    dimensions=[(0, 1), (2, 3), (0, 1), (2, 3), (0, 1), (2, 3)],
    ncols=2,
    size=1,
    show=False,
    save="_PC1_PC2_against_samples_mt_phase.pdf"
)

[<AxesSubplot: title={'center': 'orig.ident'}, xlabel='PC1', ylabel='PC2'>,
 <AxesSubplot: title={'center': 'orig.ident'}, xlabel='PC3', ylabel='PC4'>,
 <AxesSubplot: title={'center': 'pct_counts_mt'}, xlabel='PC1', ylabel='PC2'>,
 <AxesSubplot: title={'center': 'pct_counts_mt'}, xlabel='PC3', ylabel='PC4'>,
 <AxesSubplot: title={'center': 'phase'}, xlabel='PC1', ylabel='PC2'>,
 <AxesSubplot: title={'center': 'phase'}, xlabel='PC3', ylabel='PC4'>]

In [6]:
# Use nearest neighbors to construct a neighborhood graph of cells using the PCA
# dimensionality reduction
sc.pp.neighbors(gbm43_data)
sc.tl.umap(gbm43_data)
sc.pl.umap(gbm43_data, color = "orig.ident", size = 2, save="_UMAP_samples.pdf")

/opt/miniconda3/envs/human_gbm_dnapk/lib/python3.11/site-packages/scanpy/plotting/_utils.py:347: UserWarning: Matplotlib is currently using cairo, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [7]:
# Use the Leiden graph clustering method to directly cluster the neighborhood graph.
# Note that here we use a clustering resolution of 1.
sc.tl.leiden(gbm43_data, resolution=1, flavor="igraph", n_iterations=2, key_added="leiden_1.0")
sc.pl.umap(gbm43_data, color=["leiden_1.0"], legend_loc="on data", show=False, save="_leiden_res_1.0.pdf")

<AxesSubplot: title={'center': 'leiden_1.0'}, xlabel='UMAP1', ylabel='UMAP2'>

In [ ]:
# Visualize all of the potential sources of variance that we want/don't want
sc.pl.umap(
    gbm43_data,
    color=["leiden_1.0", "log1p_total_counts", "pct_counts_mt", "log1p_n_genes_by_counts", "cond", "phase"],
    wspace=0.5,
    ncols=2,
    show=False,
    save="_leiden_total_counts_mt_cond_phase.pdf"
)

[<AxesSubplot: title={'center': 'leiden_1.0'}, xlabel='UMAP1', ylabel='UMAP2'>,
 <AxesSubplot: title={'center': 'log1p_total_counts'}, xlabel='UMAP1', ylabel='UMAP2'>,
 <AxesSubplot: title={'center': 'pct_counts_mt'}, xlabel='UMAP1', ylabel='UMAP2'>,
 <AxesSubplot: title={'center': 'log1p_n_genes_by_counts'}, xlabel='UMAP1', ylabel='UMAP2'>,
 <AxesSubplot: title={'center': 'cond'}, xlabel='UMAP1', ylabel='UMAP2'>,
 <AxesSubplot: title={'center': 'phase'}, xlabel='UMAP1', ylabel='UMAP2'>]

In [ ]:
# How many different cell types are there? It seems like somewhere between 1 and 0.5 is going to be our
# optimal leiden clustering resolution.
for res in [0.02, 0.5, 0.7, 2.0]:
    sc.tl.leiden(
        gbm43_data, key_added=f"leiden_res_{res:4.2f}", resolution=res, flavor="igraph"
    )
sc.pl.umap(
    gbm43_data,
    color=["leiden_res_0.02", "leiden_res_0.50", "leiden_res_2.00", "leiden_1.0", "leiden_res_0.70"],
    legend_loc="on data",
    show=False,
    save="_leiden_resolutions.pdf"
)

[<AxesSubplot: title={'center': 'leiden_res_0.02'}, xlabel='UMAP1', ylabel='UMAP2'>,
 <AxesSubplot: title={'center': 'leiden_res_0.50'}, xlabel='UMAP1', ylabel='UMAP2'>,
 <AxesSubplot: title={'center': 'leiden_res_2.00'}, xlabel='UMAP1', ylabel='UMAP2'>,
 <AxesSubplot: title={'center': 'leiden_1.0'}, xlabel='UMAP1', ylabel='UMAP2'>,
 <AxesSubplot: title={'center': 'leiden_res_0.70'}, xlabel='UMAP1', ylabel='UMAP2'>]

In [ ]:
# Calculate marker genes for the cluster of interest
sc.tl.rank_genes_groups(
    gbm43_data,
    groupby="leiden_1.0",
    method="wilcoxon",
    use_raw=False
)
dotplot = sc.pl.rank_genes_groups_dotplot(
    gbm43_data, groupby="leiden_1.0", standard_scale="var", n_genes=5, use_raw=False, show=False, save="_top_5_markers_leiden_1.0.pdf", return_fig=True
)

categories: 0, 1, 2, etc.
var_group_labels: low, high


In [ ]:
# Since there are maybe 10 PCs that have the highest contribution, let's take a look at what they are
pc_genes_dict = {}
pca_loadings = gbm43_data.varm["PCs"]
for pc in range(10):
    pc_genes = pca_loadings[:, pc]
    top_genes = gbm43_data.var_names[np.argsort(-np.abs(pc_genes))[:10]].tolist()
    pc_genes_dict[pc] = top_genes

In [ ]:
# Plot the PC genes against UMAP space
for pc in range(3):
    genes = pc_genes_dict[pc]
    sc.set_figure_params(figsize=[3, 3])  # Increased figure size for better spacing
    sc.pl.umap(
        gbm43_data,
        color=genes,
        legend_loc="on data",
        show=False,
        save=f"_PC_dim_{pc}_top_genes.pdf"
    )

In [ ]:
# What drives the horizontal axis of variation in the current UMAP plot?
umap_1 = gbm43_data.obsm["X_umap"][:, 0]
gbm43_data.obs['umap_1_group'] = pd.cut(umap_1, bins=2, labels=['low', 'high'])
sc.tl.rank_genes_groups(gbm43_data, groupby='umap_1_group', method='t-test', use_raw=False)
sc.pl.rank_genes_groups(gbm43_data, n_genes=10, use_raw=False, show=False, save="_horizontal_UMAP_axis_top_genes.pdf")

In [ ]:
# Save the processed adata object
gbm43_data.write(
    os.path.join(DATA_DIR, "gbm43_processed.h5ad")
)

In [33]:
# Reload
gbm43_data = ad.read_h5ad(os.path.join(DATA_DIR, "gbm43_processed.h5ad"))

In [21]:
# Export the obs for manuscript purposes
umap_df = pd.DataFrame(gbm43_data.obsm['X_umap'], index=gbm43_data.obs_names, columns=['umap_x', 'umap_y'])
obs_with_umap_df = gbm43_data.obs.join(umap_df)
obs_with_umap_df.index.name = "Cell barcode"
obs_with_umap_df.to_csv("../output/manuscript/FigS2A_gbm43_perturb_seq_sc_analysis_metadata_with_umap.csv")

In [67]:
color_df = dotplot.dot_color_df
size_df = dotplot.dot_size_df
color_df.to_csv("../output/manuscript/FigS2B_rank_genes_groups_dotplot_color_df.csv", index=False)
size_df.to_csv("../output/manuscript/FigS2B_rank_genes_groups_dotplot_size_df.csv", index=False)
rank_genes_groups_info = gbm43_data.uns['rank_genes_groups']

In [127]:
# Construct a single massive dataframe with all of this information for the top 20 ranked genes for each group
groups = rank_genes_groups_info['names'].dtype.names
data_records = []
stats = ['scores', 'pvals', 'pvals_adj', 'logfoldchanges']
for group in groups:
    genes = rank_genes_groups_info['names'][group]
    for rank, gene in enumerate(genes[:20]):
        record = {'group': group, 'gene': gene, 'rank': rank}
        for stat in stats:
            record[stat] = rank_genes_groups_info[stat][group][rank]
        data_records.append(record) 
data_records = pd.DataFrame(data_records)
data_records.to_csv("../output/manuscript/FigS2B_rank_genes_groups_top20_genes_stats_pvalues_lfcs_dataframe.csv", index=False)           

#### Cell cycle regression

In [ ]:
# Note that for better or worse, cell cycle regression appeared to be a significant part of the vertical variation in the UMAP.
# Thus, we will try regressing it out. Due to downstream needs, we will not scale. In order to not regress out differences between
# proliferative and non-proliferative cells, we will not regress out S_score and G2M_score, but instead the difference between them.

# In doing so, we are constructing a dataset where we have modeled some regression coefficient for *cell_cycle_diff*. This regression
# coefficient attempts to predict gene expression based off of some baseline + B * cell_cycle_diff for each gene, solved across cells.
# We then subtract the influence of cell_cycle_diff between S and G2M phase cells from each cell for each gene based on the model.
gbm43_data.obs["cell_cycle_diff"] = gbm43_data.obs["S_score"] - gbm43_data.obs["G2M_score"]
gbm43_data_regressed = sc.pp.regress_out(gbm43_data, ["cell_cycle_diff"], copy=True)

In [ ]:
# For downstream analysis, we need positive values only
sc.pp.scale(gbm43_data_regressed)
gbm43_data_regressed.X = np.clip(gbm43_data_regressed.X, 1e-5, None)

In [ ]:
# Run PCA and clustering
sc.tl.pca(gbm43_data_regressed)

In [ ]:
# How many PCs should we consider?
sc.pl.pca_variance_ratio(gbm43_data_regressed, n_pcs=50, log=True)

In [ ]:
sc.pl.pca(
    gbm43_data_regressed,
    color=["orig.ident", "orig.ident", "pct_counts_mt", "pct_counts_mt", "phase", "phase"],
    dimensions=[(0, 1), (2, 3), (0, 1), (2, 3), (0, 1), (2, 3)],
    ncols=2,
    size=1,
)

In [ ]:
# Use nearest neighbors to construct a neighborhood graph of cells using the PCA
# dimensionality reduction
sc.pp.neighbors(gbm43_data_regressed)
sc.tl.umap(gbm43_data_regressed)
sc.pl.umap(gbm43_data_regressed, color = "orig.ident", size = 2)

In [ ]:
# Use the Leiden graph clustering method to directly cluster the neighborhood graph.
# Note that here we use a clustering resolution of 1.
sc.tl.leiden(gbm43_data_regressed, resolution=1, flavor="igraph", n_iterations=2, key_added="leiden_1.0")
sc.pl.umap(gbm43_data_regressed, color=["leiden_1.0", "phase"], legend_loc="on data")

In [ ]:
# Since there are maybe 10 PCs that have the highest contribution, let's take a look at what they are
pc_genes_dict = {}
pca_loadings = gbm43_data_regressed.varm["PCs"]
for pc in range(10):
    pc_genes = pca_loadings[:, pc]
    top_genes = gbm43_data_regressed.var_names[np.argsort(-np.abs(pc_genes))[:10]].tolist()
    pc_genes_dict[pc] = top_genes

In [ ]:
# Plot the PC genes against UMAP space
for pc in range(3):
    genes = pc_genes_dict[pc]
    sc.set_figure_params(figsize=[3, 3])  # Increased figure size for better spacing
    sc.pl.umap(
        gbm43_data_regressed,
        color=genes,
        legend_loc="on data",
    )

In [ ]:
# What drives the horizontal axis of variation in the current UMAP plot?
umap_1 = gbm43_data_regressed.obsm["X_umap"][:, 0]
gbm43_data_regressed.obs['umap_1_group'] = pd.cut(umap_1, bins=2, labels=['low', 'high'])
sc.tl.rank_genes_groups(gbm43_data_regressed, groupby='umap_1_group', method='t-test', use_raw=False)
sc.pl.rank_genes_groups(gbm43_data_regressed, n_genes=10, use_raw=False)

In [ ]:
gbm43_data.write(
    os.path.join(DATA_DIR, "gbm43_processed_cell_cycle_regressed.h5ad")
)